# Permutation tests based on ranks, and calibrating parametric tests using permutations

This notebook explores some common nonparametric tests based on ranks, and shows that many parametric tests can be calibrated nonparametrically under the null.

## Notation

Ranks



## The two-sample problem

### Testing for a difference in location

### Testing for a difference in dispersion

### Testing for a difference in distribution

## The $k$-sample problem

### Testing for differences in location